# Stochastic Programming Example Using Jump and DSP

## Farmer Crop Optimization

## Model

#### Here, we setup the data relevant to the farmer model.  We set 3 possible scenarios of equal probability, and then three sets of crops to plant with the associated costs of planting each crop.  We also define the total budget.  The second stage model data adds the possiblity to purchase and sell crops with corresponding purchase and sell prices.

This guide illustrates how to use DSP for solving the following farmer problem.

\begin{align}
\text{minimize} \qquad & 150x_1+230x_2+260x_3 + \sum_{s=1:3}\frac{1}{3}(238y_{1s}+210y_{2s}−170y_{3s}−150y_{4s}−36y_{5s}−10y_{6s}) \\
 \text{subject to} \quad \quad & x_1+x_2+x_3 \leq 500 \\
  \quad \quad & T_{1s}x_1+y_{1s}−y_{3s} \geq 200 \quad s=1,2,3 \\
  \quad \quad & T_{2s}x_2+y_{2s}−y_{4s} \geq 240 \quad s=1,2,3 \\
  \quad \quad & T_{3s}x_3+y_{5s}−y_{6s} \geq 0 \quad s=1,2,3 \\
  \quad \quad & y_{5s} \leq 6000 \quad s=1,2,3 \\
 \qquad \qquad & x_1,x_2,x_3 \geq 0, y_{1s},y_{2s},y_{3s},y_{4s},y_{5s},y_{6s}  \geq 0  \quad s=1,2,3\\
 \qquad \qquad & x,y \in \mathbb{R}
\end{align}

where $T_{is}$ is an (i,s)(i,s)-element of matrix
$$
T:=  \begin{bmatrix}
    3.0       & 3.6 & 24.0   \\
    2.5       & 3.0 & 20.0  \\
    2.0       & 2.4 & 16.0 
\end{bmatrix}
$$



In [14]:
Libdl.find_library(["libDsp"])

"libDsp"

In [2]:
using JuMP, Dsp

In [3]:
NS = 3;                        # number of scenarios
probability = [1/3, 1/3, 1/3]; # probability

CROPS  = 1:3 # set of crops (wheat, corn and sugar beets, resp.)
PURCH  = 1:2 # set of crops to purchase (wheat and corn, resp.)
SELL   = 1:4 # set of crops to sell (wheat, corn, sugar beets under 6K and those over 6K)

Cost     = [150 230 260]   # cost of planting crops
Budget   = 500             # budget capacity
Purchase = [238 210];      # purchase price
Sell     = [170 150 36 10] # selling price
Yield    = [3.0 3.6 24.0;
            2.5 3.0 20.0;
            2.0 2.4 16.0]
Minreq   = [200 240 0]     # minimum crop requirement

# JuMP model
m = Model(NS)

@variable(m, x[i=CROPS] >= 0, Int)
@objective(m, Min, sum(Cost[i] * x[i] for i=CROPS))
@constraint(m, const_budget, sum(x[i] for i=CROPS) <= Budget)

for s in blockids()
    blk = Model(m, s, probability[s])

    @variable(blk, y[j=PURCH] >= 0)
    @variable(blk, w[k=SELL] >= 0)

    @objective(blk, Min, sum(Purchase[j] * y[j] for j=PURCH) - sum(Sell[k] * w[k] for k=SELL))

    @constraint(blk, const_minreq[j=PURCH], Yield[s,j] * x[j] + y[j] - w[j] >= Minreq[j])
    @constraint(blk, const_minreq_beets, Yield[s,3] * x[3] - w[3] - w[4] >= Minreq[3])
    @constraint(blk, const_aux, w[3] <= 6000)
end

In [12]:
# Dsp solve types
solve_types = [:Dual, :Benders, :Extensive]

# Default parameter file
myparam = joinpath(dirname(@__FILE__),"DSP/parameters/default.txt")

status = solve(m, solve_type = solve_types[2], param = myparam)



Finding a good lower bound using Dual Decomposition...
CPLEX is not available for QP solve.

DUAL DECOMPOSITION ITERATION INFORMATION:
* master   = objective function value of the master problem.
* primobj  = best primal objective function value.
* dualobj  = best dual objective function value.
* a.gap(%) = Approximate gap between master and dualobj.
* d.gap(%) = Duality gap between primobj and dualobj.
* times    = wall clock time in seconds.

  iter         master        primobj        dualobj  a.gap(%)  d.gap(%)    time
 D   0  -1.151370e+05          Large  -1.154000e+05      0.23    100.00     0.0
 D   1  -1.146110e+05          Large  -1.151370e+05      0.46    100.00     0.0
 D   2  -1.135590e+05          Large  -1.146110e+05      0.93    100.00     0.0
 D   3  -1.114550e+05          Large  -1.135590e+05      1.89    100.00     0.1
 D   4  -1.093510e+05          Large  -1.126556e+05      3.02    100.00     0.1
     5  -1.098333e+05          Large  -1.126556e+05      2.57    100.00

:Optimal

In [7]:
@show getprimobjval()
@show getdualobjval()
@show getprimvalue()
@show getdualvalue()

getprimobjval() = -108389.99999999996
getdualobjval() = -108389.99999999996
getprimvalue() = [170.0, 80.0, 250.0, 0.0, 0.0, 310.0, 48.0, 6000.0, 4.54747e-12, 0.0, 2.78533e-12, 225.0, 0.0, 5000.0, 0.0, 0.0, 48.0, 140.0, 0.0, 4000.0, 0.0]
getdualvalue() = Float64[]


0-element Array{Float64,1}

In [8]:
getvalue(x)

x: 1 dimensions:
[1] = 170.00000000000077
[2] = 79.99999999999908
[3] = 250.0000000000002

In [13]:
getblocksolution(m)

3-element Array{Any,1}:
 [0.0, 2.78533e-12, 225.0, 0.0, 5000.0, 0.0] 
 [0.0, 48.0, 140.0, 0.0, 4000.0, 0.0]        
 [0.0, 0.0, 310.0, 48.0, 6000.0, 4.54747e-12]